In [16]:
from sentence_transformers import SentenceTransformer
import sqlite3
from typing import Generator
from pathlib import Path
import numpy as np

DATABASE = Path("../../db/book5.db")
TABLE_NAME = "book5"
model = SentenceTransformer("all-MiniLM-L6-v2")

/home/art3mis/projects/prophetic-strategies/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
def check_db_col_names(db, table):
    with sqlite3.connect(db) as conn:
        cursor = conn.cursor()
        cursor.execute(f"""PRAGMA table_info({table});""")
        print(cursor.fetchall())


check_db_col_names(DATABASE, TABLE_NAME)

[(0, 'book', 'TEXT', 0, None, 0), (1, 'sentence', 'INTEGER', 0, None, 0), (2, 'chapter', 'INTEGER', 0, None, 0), (3, 'sentence_chapter', 'INTEGER', 0, None, 0), (4, 'content', 'TEXT', 0, None, 0), (5, 'embedding', 'TEXT', 0, None, 0), (6, 'embedding_minilm', 'BLOB', 0, None, 0)]


In [18]:
def alter_table():
    with sqlite3.connect(DATABASE) as conn:
        cursor = conn.cursor()
        cursor.execute(
            """ALTER TABLE book5
            DROP COLUMN embedding_minilm;
        """
        )
        cursor.execute(
            """
            ALTER TABLE book5
            ADD COLUMN embedding_minilm TEXT
        """
        )

        conn.commit()

In [ ]:
# alter_table()

In [ ]:
check_db_col_names(DATABASE, TABLE_NAME)

In [19]:
def add_embedding_to_db(db, table_name):
    with sqlite3.connect(DATABASE) as conn:
        cursor = conn.cursor()
        cursor.execute(f"""SELECT sentence, content FROM {TABLE_NAME}""")
        rows = cursor.fetchall()

        for row in rows:
            row_id = row[0]
            sentence = row[1]

            embedding = model.encode(sentence)

            cursor.execute(
                f"""
                UPDATE {TABLE_NAME}
                SET embedding_minilm = ?
                WHERE sentence = ?
            """,
                (embedding, row_id),
            )

            conn.commit()

In [ ]:
# add_embedding_to_db(DATABASE, TABLE_NAME)

In [12]:
from sentence_transformers import SentenceTransformer
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np


def similarity_minilm(question):
    DATABASE = Path("../../db/book5.db")
    TABLE_NAME = "book5"
    model = SentenceTransformer("all-MiniLM-L6-v2")

    qv = model.encode(question)
    df = get_wv(DATABASE, TABLE_NAME)
    similarities = model.similarity([qv], (df["embedding_minilm"]))
    df["similarity"] = similarities


def get_wv(db: Path, table_name: str) -> pd.DataFrame:
    with sqlite3.connect(db) as conn:
        rows = pd.read_sql(f"""SELECT * FROM {table_name}""", conn)
        return rows


In [20]:
DATABASE = Path("../../db/book5.db")
TABLE_NAME = "book5"
df = get_wv(DATABASE, TABLE_NAME)
df

,book,sentence,chapter,sentence_chapter,content,embedding,embedding_minilm
0,Book V - The Formations of the Unconscious (19...,1,1,1,This year we have taken the formations of the ...,b'\x00\x00\x00\xc0bZ\x97\xbf\x00\x00\x00\x80\x...,None
1,Book V - The Formations of the Unconscious (19...,2,1,2,Those of you - I think it was the majority - w...,"b'\x00\x00\x00`""\xc5\x8d?\x00\x00\x00 \xe7\xb5...",None
2,Book V - The Formations of the Unconscious (19...,3,1,3,A certain number of you - I am only expressing...,b'\x00\x00\x00@\xe8J\x94?\x00\x00\x00 \x0c\xd3...,None
3,Book V - The Formations of the Unconscious (19...,4,1,4,Those who have had the courage to do so will b...,b'\x00\x00\x00@f\xd9\x98?\x00\x00\x00`:\\{?\x0...,None
4,Book V - The Formations of the Unconscious (19...,5,1,5,In a way it is a modest enough ambition for me...,b'\x00\x00\x00\xc0(D\xad?\x00\x00\x00 \xd1\xd0...,None
...,...,...,...,...,...,...,...
3755,Book V - The Formations of the Unconscious (19...,3756,28,118,Experience proves that we have the most contra...,b'\x00\x00\x00\x80\xe0\xe7\xa0?\x00\x00\x00\x8...,None
3756,Book V - The Formations of the Unconscious (19...,3757,28,119,I think that in fact these objections which ar...,b'\x00\x00\x00\x80d\xfc\x9d?\x00\x00\x00`\xb6C...,None
3757,Book V - The Formations of the Unconscious (19...,3758,28,120,"In fact, I believe that if something allows us...",b'\x00\x00\x00 \x98\xcd\x97?\x00\x00\x00 \x8f\...,None
3758,Book V - The Formations of the Unconscious (19...,3759,28,121,It is at this level that the Christian command...,b'\x00\x00\x00\x80!\xd1A\xbf\x00\x00\x00\x00\x...,None


In [13]:

similarity_minilm("freud")

/home/art3mis/projects/prophetic-strategies/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.